# Prepare from bootstrap paths

Now we will use the initial trajectories we obtained from bootstrapping to run an MSTIS simulation. This will show both how objects can be regenerated from storage and how regenerated equivalent objects can be used in place of objects that weren't stored.

Tasks covered in this notebook:
* Loading OPS objects from storage
* Ways of assigning initial trajectories to initial samples
* Setting up a path sampling simulation with various move schemes
* Visualizing trajectories while the path sampling is running

In [1]:
%matplotlib inline
import openpathsampling as paths
import numpy as np
import math

In [2]:
# Imports for plotting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from matplotlib.legend_handler import HandlerLine2D

### Loading things from storage

First we'll reload some of the stuff we stored before. Of course, this starts with opening the file.

In [3]:
old_store = paths.AnalysisStorage("ala_mstis_bootstrap.nc")

A lot of information can be recovered from the old storage, and so we don't have the recreate it. However, we did not save our network, so we'll have to create a new one. Since the network creates the ensembles, that means we will have to translate the trajectories from the old ensembles to new ensembles.

In [4]:
print "PathMovers:", len(old_store.pathmovers)
print "Engines:", len(old_store.engines)
print "Samples:", len(old_store.samples)
print "Ensembles:", len(old_store.ensembles)
print "SampleSets:", len(old_store.samplesets)
print "Snapshots:", len(old_store.snapshots)
print "Networks:", len(old_store.networks)

PathMovers: 186
Engines: 2
Samples: 29
Ensembles: 351
SampleSets: 1
Snapshots: 4210
Networks: 1


Loading from storage is very easy. Each store is a list. We take the 0th snapshot as a template (it doesn't actually matter which one) for the next storage we'll create. There's only one engine stored, so we take the only one.

In [5]:
template = old_store.snapshots[0]

In [6]:
engine = old_store.engines['default']

In [7]:
mstis = old_store.networks[0]

Now we need to set up real trajectories that we can use for each of these. We can start by loading the stored sample set.

In [8]:
sset = paths.SampleSet(old_store.samplesets[0])

In [10]:
sset.sanity_check()
assert(len(sset)==29)

## Equilibration

In molecular dynamics, you need to equilibrate if you don't start with an equilibrium frame (e.g., if you start with solvent molecules on a grid, your system should equilibrate before you start taking statistics). Similarly, if you start with a set of paths which are far from the path ensemble equilibrium, you need to equilibrate. This could either be because your trajectories are not from the real dynamics (generated with metadynamics, high temperature, etc.) or because your trajectories are not representative of the path ensemble (e.g., if you put transition trajectories into all interfaces).

As with MD, running equilibration can be the same process as running the total simulation. However, in path sampling, it doesn't have to be: we can equilibrate without replica exchange moves or path reversal moves, for example. In the example below, we create a `MoveScheme` that only includes shooting movers.

In [11]:
equil_scheme = paths.MoveScheme(mstis)
import openpathsampling.analysis.move_strategy as strat
equil_scheme.append([
        strat.OneWayShootingStrategy(engine=engine), 
        strat.OrganizeByMoveGroupStrategy()
    ])

In [12]:
equilibration = paths.PathSampling(
    storage=None,
    globalstate=sset,
    move_scheme=equil_scheme
)

In [13]:
equilibration.run(20)

Working on Monte Carlo cycle number 20.
DONE! Completed 20 Monte Carlo cycles.


In [20]:
sset = equilibration.globalstate

In [22]:
import openpathsampling as paths

In [23]:
storage = paths.storage.Storage("weina_pre.nc", "w")

In [24]:
storage.save(template)

(store.snapshots[BaseSnapshot],
 2,
 UUID('52e49e51-4765-11e6-a326-000000000042'))

In [25]:
storage.snapshots.variables['uuid'][:]

array([u'52e49e51-4765-11e6-a326-000000000042'], dtype=object)

In [7]:
tt = old_store.trajectories[0]

In [8]:
for snap in tt.iter_proxies():
    i = snap._idx
    
    print snap._store is old_store.snapshots, i, old_store.snapshots.pos(i), storage.snapshots.pos(i)

True 52e49e51-4765-11e6-a326-00000000000c 4 None
True 52e49e51-4765-11e6-a326-00000000013a 6 None
True 52e49e51-4765-11e6-a326-000000000154 8 None
True 52e49e51-4765-11e6-a326-00000000016e 10 None
True 52e49e51-4765-11e6-a326-000000000188 12 None
True 52e49e51-4765-11e6-a326-0000000001a2 14 None
True 52e49e51-4765-11e6-a326-0000000001bc 16 None
True 52e49e51-4765-11e6-a326-0000000001d6 18 None


In [9]:
storage.trajectories.save(tt)

UUID('52e49e51-4765-11e6-a326-000000000120')

In [26]:
storage.save(sset)
storage.save(mstis)
storage.save(engine)

(store.engines[DynamicsEngine],
 11,
 UUID('52e49e51-4765-11e6-a326-000000000010'))

In [27]:
for tag in ['states', 'state_centers', 'state_letter', 'interface_levels']:
    storage.tag[tag] = old_store.tag[tag]

In [28]:
storage.tag['engine'] = engine
storage.tag['initial_sampleset'] = sset
storage.tag['mstis'] = mstis

In [29]:
import os

In [30]:
print 'filesize : %d MB' % (os.stat('weina_pre.nc').st_size / 1024 / 1024)

filesize : 37 MB


In [31]:
storage.close()